In [2]:
# HERE ARE FOR COLAB RUNNING 

# Install a Drive FUSE wrapper.
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse


# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()
# Generate creds for the Drive FUSE library.
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 131323 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.1-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [3]:
# HERE ARE FOR COLAB RUNNING 

# Create a directory and mount Google Drive using that directory.
!mkdir -p drive
!google-drive-ocamlfuse drive
!ls /content/drive/colab/CS155_MiniProject_3

2_notebook.ipynb     pre_processing.py		     RNN_char_word.ipynb
build_rhyme_dict.py  project3			     RNN.py
HMM_helper.py	     __pycache__		     split_poem.py
HMM.py		     README.md			     syllable_dictionary.py
hw6_hmm_codes_sol    rhyme_HMM_shakespeare.ipynb     test_module.ipynb
naive_HMM.ipynb      rhyme_HMM_spenser_shakes.ipynb  Utility.py


In [4]:
# HERE ARE FOR COLAB RUNNING 

!pip install keras
!pip install numpy
!pip install nltk
!pip install pyphen

    100% |████████████████████████████████| 3.0MB 11.6MB/s 


In [0]:
# HERE ARE FOR COLAB RUNNING 
import sys
#sys.path.append('drive/My Drive/colab')
sys.path.append('/content/drive/colab/CS155_MiniProject_3')
#'drive/My Drive/colab/pre_processing.py'

In [6]:
# HERE ARE FOR GPU RUNNING
import tensorflow as tf
tf.test.gpu_device_name()
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

Using TensorFlow backend.


['/job:localhost/replica:0/task:0/device:GPU:0']

In [7]:
# CODES START HERE

import numpy as np
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers.core import Dense, Activation, Dropout, Lambda
from keras.layers.recurrent import LSTM
from keras.layers import CuDNNLSTM
from keras.preprocessing.sequence import pad_sequences
from pre_processing import *

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.


In [0]:
def load_text_as_string(filepath):
    poems_ori = load_shakespeare_sentences(filepath)
    poems = lower_case(poems_ori)
    text = ''
    #print(poems)
    for line in poems:
            text += line + '\n'
    return text

def train_rnn(X, y, temperature, batch_size, epochs):
    model = Sequential()
    model.add(CuDNNLSTM(200, return_sequences=False, input_shape=(len(X[0]), len(X[0][0]))))
    model.add(Dropout(0.2))
    # model.add(LSTM(200, return_sequences=False))
    # model.add(Dropout(0.2))
    model.add(Dense(len(X[0][0])))
    model.add(Activation('softmax'))
    model.add(Lambda(lambda x: x / temperature))

    model.compile(loss='categorical_crossentropy', optimizer='adam')
    #print(model.metrics_names)
    model.fit(X, y, batch_size = batch_size, epochs = epochs )
    
    error_in = model.evaluate(X, y, batch_size = batch_size)
    return model, error_in

def train_rnn_word(X, y, temperature, batch_size, epochs):
    model = Sequential()
    model.add(CuDNNLSTM(200, return_sequences=False, input_shape=(len(X[0]), len(X[0][0]))))
    model.add(Dropout(0.2))
    # model.add(LSTM(200, return_sequences=False))
    # model.add(Dropout(0.2))
    model.add(Dense(len(X[0][0])))
    model.add(Activation('softmax'))
    model.add(Lambda(lambda x: x / temperature))

    model.compile(loss='categorical_crossentropy', optimizer='adam')
    #print(model.metrics_names)
    model.fit(X, y, batch_size = batch_size, epochs = epochs )
    
    error_in = model.evaluate(X, y, batch_size = batch_size)
    return model, error_in

# for char training
def generate_train_data(text, step):
    char_list = list(set(text))
    char_dict = {}
    index = 0
    for c in char_list:
        char_dict[c] = index
        index += 1
    char_num = len(char_dict)
    
    seq_length = 40
    X_train_string = []
    Y_train_string = []
    for i in range(0, len(text)-seq_length, step):
        sentence = text[i:i+seq_length]
        next_char = text[i+seq_length]
        X_train_string.append(sentence)
        Y_train_string.append(next_char)    
    
    X_train_encode = []
    Y_train_encode = []
    for sentence in X_train_string:
        new_sentence = []
        for c in sentence:
            new_sentence.append(char_dict[c])
        X_train_encode.append(new_sentence)
    for next_char in Y_train_string:
        Y_train_encode.append(char_dict[next_char])
    
    X_train = np.array([to_categorical(seq, num_classes = char_num) for seq in X_train_encode])
    Y_train = to_categorical(Y_train_encode, num_classes = char_num)
    
    return X_train, Y_train, char_dict, char_num

# def generate_train_data_word(text, step):
# #     print("**********************")
# #     print(text)
# #     print("**********************")
#     word_list = pre_tokenize_full(text)
#     word_set = set()
#     for w in word_list:
# 	     word_set.add(w)
#     index = 0
#     word_dict = dict.fromkeys(word_set, 0)
#     #print(word_dict)
#     for key in word_dict.keys():
#         word_dict[key] = index
#         index+=1
#     word_num = len(word_dict)

#     seq_length = 10
#     X_train_string = []
#     Y_train_string = []
#     for i in range(0, len(word_list)-seq_length, step):
#         sentence = word_list[i:i+seq_length]
#         next_word = word_list[i+seq_length]
#         X_train_string.append(sentence)
#         Y_train_string.append(next_word)

#     X_train_encode = []
#     Y_train_encode = []
#     for sentence in X_train_string:
#         new_sentence = []
#         for w in sentence:
#             new_sentence.append(word_dict[w])
#         X_train_encode.append(new_sentence)
#     for next_w in Y_train_string:
#         Y_train_encode.append(word_dict[next_w])

#     X_train = np.array([to_categorical(seq, num_classes = word_num) for seq in X_train_encode])
#     Y_train = to_categorical(Y_train_encode, num_classes = word_num)

#     return X_train, Y_train, word_dict, word_num

# for word training
def generate_train_data_word(filename, step):
    sentences = load_shakespeare_sentences(filename)
    word_list_raw = []
    word_list = []
    for sentence in sentences:
        word_sublist = pre_tokenize_full(sentence)
        word_sublist.append("\n")
        word_list_raw += word_sublist
    for word in word_list_raw:
        word_list.append(word.lower())
        
    word_set = set()
    for w in word_list:
        word_set.add(w)
    index = 0
    word_dict = dict.fromkeys(word_set, 0)
    #print(word_dict)
    for key in word_dict.keys():
        word_dict[key] = index
        index+=1
    word_num = len(word_dict)

    seq_length = 10
    X_train_string = []
    Y_train_string = []
    for i in range(0, len(word_list)-seq_length, step):
        sentence = word_list[i:i+seq_length]
        next_word = word_list[i+seq_length]
        X_train_string.append(sentence)
        Y_train_string.append(next_word)

    X_train_encode = []
    Y_train_encode = []
    for sentence in X_train_string:
        new_sentence = []
        for w in sentence:
            new_sentence.append(word_dict[w])
        X_train_encode.append(new_sentence)
    for next_w in Y_train_string:
        Y_train_encode.append(word_dict[next_w])

    X_train = np.array([to_categorical(seq, num_classes = word_num) for seq in X_train_encode])
    Y_train = to_categorical(Y_train_encode, num_classes = word_num)

    return X_train, Y_train, word_dict, word_num

# for char generate
def generate_sequence(model, char_dict, seed_text, num_chars, max_prob):
    char_dict_inverse = {}
    for key, val in char_dict.items():
        char_dict_inverse[val] = key

    # initlization
    new_text = seed_text
    for char in range(num_chars):
        sequence = [char_dict[char] for char in new_text]
        sequence = pad_sequences([sequence], maxlen=40)
        encoded_seq = to_categorical(sequence, num_classes=len(char_dict))

        if max_prob:
            pred_class = model.predict_classes(encoded_seq)
            #print("USE MAX_PROB GENERATION!")
            # print("************************* pred max! *************************")
            # print("max_prob pred_class = ", pred_class)
            # print("************************* pred max! end *************************")
        else:
            pred_classes = model.predict(encoded_seq)[0]
            #print("NOT USE MAX_PROB GENERATION! USE PROB_DISTRIBUTION INSTEAD!")
            # print("************************* pred no max! *************************")
            # print("no max_prob pred_classes = ", pred_classes)
            # print("************************* pred no max! end *************************")

            normalized_pred_classes = []
            sum_pred = sum(pred_classes)

            for predict in pred_classes:
                normalized_pred_classes.append(predict / sum_pred)

            pred_class = np.random.choice(range(len(normalized_pred_classes)), p=normalized_pred_classes)
            # print("************************* pred no max! *************************")
            # print("no max_prob pred_class = ", pred_class)
            # print("************************* pred no max! end *************************")

        new_char = char_dict_inverse[pred_class]
        new_text += new_char

    return new_text

# for word generate
def generate_sequence_word(model, word_dict, seed_text, num_words, max_prob):
    word_dict_inverse = {}
    for key, val in word_dict.items():
        word_dict_inverse[val] = key

    # initlization
    new_text = seed_text
    # new_text_words = new_text.split(" ")
    new_text_words = pre_tokenize_full(new_text)
    new_text_words.append("\n")
    
    for word in range(num_words):
        sequence = [word_dict[word] for word in new_text_words]
        sequence = pad_sequences([sequence], maxlen=10)
        encoded_seq = to_categorical(sequence, num_classes=len(word_dict))

        if max_prob:
            pred_class = model.predict_classes(encoded_seq)
            
            # print("************************* pred max! *************************")
            # print("max_prob pred_class = ", pred_class)
            # print("************************* pred max! end *************************")
        else:
            pred_classes = model.predict(encoded_seq)[0]
            # print("************************* pred no max! word *************************")
            # print("no max_prob pred_classes = ", pred_classes)
            # print("************************* pred no max! word end *************************")

            normalized_pred_classes = []
            sum_pred = sum(pred_classes)

            for predict in pred_classes:
                normalized_pred_classes.append(predict / sum_pred)

            pred_class = np.random.choice(range(len(normalized_pred_classes)), p=normalized_pred_classes)
            # print("************************* pred no max! word_Normalize *************************")
            # print("no max_prob pred_class = ", pred_class) 
            # print("************************* pred no max! word_Normaliz end *************************")

        new_word = word_dict_inverse[pred_class]
        new_text_words.append(new_word)

    return " ".join(new_text_words)


# def rnn(filepath):
# 	#filepath = './project3/data/shakespeare.txt'
# 	text = load_text_as_string(filepath)
# 	step = 5
# 	temperature = 1.5
# 	batch_size = 32
# 	epochs = 10
# 	seed_text = "shall i compare thee to a summer's day?\n"
# 	num_chars = 3000

# 	X_train, Y_train, char_dict, char_num = generate_train_data(text, step)
# 	print(char_dict)

# 	model, error_in = train_rnn(X_train, Y_train, temperature, batch_size, epochs)

# 	# save model
# 	model.save('rnn_bs'+str(batch_size)+'_epoch'+str(epochs)+'.h5')

# 	new_text = generate_sequence(model, char_dict, seed_text, num_chars)

# 	return error_in, new_text

def rnn(filepath, temp, epochs_num):
    text = load_text_as_string(filepath)
    step = 5
    batch_size = 32
    temperature = temp
    epochs = epochs_num
    seed_text = "shall i compare thee to a summer's day?\n"
    num_chars = 3000

    X_train, Y_train, char_dict, char_num = generate_train_data(text, step)
    #print(char_dict)

    model, error_in = train_rnn(X_train, Y_train, temperature, batch_size, epochs)

    # save model
    model.save('rnn_bs'+str(batch_size)+'_epoch'+str(epochs)+'_temp'+str(temperature)+'.h5')

    new_text = generate_sequence(model, char_dict, seed_text, num_chars, False)

    return error_in, new_text

def rnn_word(filepath, temp, epochs_num):
    text = load_text_as_string(filepath)
    #step = 1 #first block
    step = 2 #following
    batch_size = 32
    temperature = temp
    epochs = epochs_num
    seed_text = "shall i compare thee to a summer's day?\n"
    num_words = 400

    X_train, Y_train, word_dict, word_num = generate_train_data_word(filepath, step)
    #print(word_dict)
    model, error_in_word = train_rnn_word(X_train, Y_train, temperature, batch_size, epochs)
    # save model
    model.save('rnn_bs'+str(batch_size)+'_epoch'+str(epochs)+'_temp'+str(temperature)+"_word"+'.h5')
    new_text_word = generate_sequence_word(model, word_dict, seed_text, num_words, False)

    return error_in_word, new_text_word

def load_model_from_file(model, filepath):
    print("************************* load_model_from_file *************************")
    print("I will use the loaded model!!! ")
    print("************************* load_model_from_file End *************************")
    text = load_text_as_string(filepath)
    step = 3
    seed_text = "shall i compare thee to a summer's day?\n"
    num_chars = 3000

    X_train, Y_train, char_dict, char_num = generate_train_data_word(filepath, step)
    new_text = generate_sequence(model, char_dict, seed_text, num_chars)
    return new_text


# if __name__ == '__main__':
# 	filepath = './project3/data/shakespeare.txt'
# 	temp = 1.5
# 	epochs_num = 200
# 	error_in_word, new_text_word = rnn_word(filepath, temp, epochs_num)
# 	print("************************* Final *************************")
# 	print("Error_In = ", error_in_word)
# 	print("new text: ", new_text_word)
# 	print("************************* Final End *************************")

	# filename = 'rnn_bs32_epoch10.h5'
	# model_2 = load_model(filename)
	# model_2.compile(loss='categorical_crossentropy', optimizer='adam')
	# new_text_2 = load_model_from_file(model_2, filepath)
	# print("************************* Load Model *************************")
	# print("new text: ", new_text_2)
	# print("************************* Load Model End *************************")




In [0]:
def running(filepath, temp, epochs_num):
    error_in_word_list = []
    new_text_word_list = []
    for epoch in epochs_num:
        for t in temp:
            error_in_word, new_text_word = rnn_word(filepath, t, epoch)
            error_in_word_list.append(error_in_word)
            new_text_word_list.append(new_text_word)
            print("************************* Final *************************")
            print("Current epoch number = ", epoch)
            print("Current Temperature = ", t)
            print("Error_In_Word = ", error_in_word, ", Perplexity metrics = ", np.exp(error_in_word))
            print("*************************Generate Word Text *************************")
            print("new text word: ", new_text_word)
            print("*************************Generate Word Text End*************************")
    return error_in_word_list, new_text_word_list

In [0]:
def running_char(filepath, temp, epochs_num):
    error_in_char_list = []
    new_text_char_list = []
    for epoch in epochs_num:
        for t in temp:
            error_in_char, new_text_char = rnn(filepath, t, epoch)
            error_in_char_list.append(error_in_char)
            new_text_char_list.append(new_text_char)
            print("************************* Final *************************")
            print("Current epoch number = ", epoch)
            print("Current Temperature = ", t)
            print("Error_In_Char = ", error_in_char, ", Perplexity metrics = ", np.exp(error_in_char))
            print("*************************Generate Char Text *************************")
            print("new text word: ", new_text_char)
            print("*************************Generate Char Text End*************************")
    return error_in_char_list, new_text_char_list

In [0]:
filepath = '/content/drive/colab/CS155_MiniProject_3/project3/data/shakespeare.txt'

In [21]:
# char based
temp_char = [1.5, 0.75, 0.25]
epochs = [50]
error_in_char_list_1, new_test_char_list_1 =  running_char(filepath, temp_char, epochs)

Epoch 1/50
18727/18727 [==============================] - 12s 631us/step - loss: 2.8215
Epoch 2/50
18727/18727 [==============================] - 11s 569us/step - loss: 2.3497
Epoch 3/50
18727/18727 [==============================] - 11s 583us/step - loss: 2.2065
Epoch 4/50
18727/18727 [==============================] - 11s 595us/step - loss: 2.0994
Epoch 5/50
18727/18727 [==============================] - 11s 593us/step - loss: 2.0234
Epoch 6/50
18727/18727 [==============================] - 11s 583us/step - loss: 1.9523
Epoch 7/50
18727/18727 [==============================] - 10s 529us/step - loss: 1.8752
Epoch 8/50
18727/18727 [==============================] - 10s 536us/step - loss: 1.8134
Epoch 9/50
18727/18727 [==============================] - 11s 564us/step - loss: 1.7516
Epoch 10/50
18727/18727 [==============================] - 11s 580us/step - loss: 1.6928
Epoch 11/50
18727/18727 [==============================] - 11s 582us/step - loss: 1.6319
Epoch 12/50
18727/18727 [=====

In [22]:
# char based
temp_char = [1.5, 0.75, 0.25]
epochs = [100]
error_in_char_list_2, new_test_char_list_2 =  running_char(filepath, temp_char, epochs)

Epoch 1/100
18727/18727 [==============================] - 10s 530us/step - loss: 2.8203
Epoch 2/100
18727/18727 [==============================] - 9s 463us/step - loss: 2.3529
Epoch 3/100
18727/18727 [==============================] - 9s 478us/step - loss: 2.2046
Epoch 4/100
18727/18727 [==============================] - 10s 514us/step - loss: 2.0960
Epoch 5/100
18727/18727 [==============================] - 9s 503us/step - loss: 2.0138
Epoch 6/100
18727/18727 [==============================] - 9s 469us/step - loss: 1.9389
Epoch 7/100
18727/18727 [==============================] - 9s 473us/step - loss: 1.8671
Epoch 8/100
18727/18727 [==============================] - 10s 512us/step - loss: 1.8011
Epoch 9/100
18727/18727 [==============================] - 10s 516us/step - loss: 1.7338
Epoch 10/100
18727/18727 [==============================] - 9s 470us/step - loss: 1.6735
Epoch 11/100
18727/18727 [==============================] - 9s 465us/step - loss: 1.6123
Epoch 12/100
18727/18727 [

In [0]:
# char based
temp_char = [1.5, 0.75, 0.25]
epochs = [150]
error_in_char_list_3, new_test_char_list_3 =  running_char(filepath, temp_char, epochs)

Epoch 1/150
18727/18727 [==============================] - 10s 545us/step - loss: 2.8237
Epoch 2/150
18727/18727 [==============================] - 9s 480us/step - loss: 2.3532
Epoch 3/150
18727/18727 [==============================] - 9s 468us/step - loss: 2.2065
Epoch 4/150
18727/18727 [==============================] - 9s 468us/step - loss: 2.1013
Epoch 5/150
18727/18727 [==============================] - 9s 462us/step - loss: 2.0246
Epoch 6/150
18727/18727 [==============================] - 9s 463us/step - loss: 1.9472
Epoch 7/150
18727/18727 [==============================] - 9s 465us/step - loss: 1.8793
Epoch 8/150
18727/18727 [==============================] - 9s 487us/step - loss: 1.8065
Epoch 9/150
18727/18727 [==============================] - 11s 590us/step - loss: 1.7465
Epoch 10/150
18727/18727 [==============================] - 11s 597us/step - loss: 1.6843
Epoch 11/150
18727/18727 [==============================] - 11s 602us/step - loss: 1.6164
Epoch 12/150
18727/18727 [

In [12]:
# char based
# the above loss connection, here re-run the codes
temp_char = [0.25]
epochs = [150]
error_in_char_list_3, new_test_char_list_3 =  running_char(filepath, temp_char, epochs)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch 1/150
18727/18727 [==============================] - 12s 653us/step - loss: 2.8295
Epoch 2/150
18727/18727 [==============================] - 9s 480us/step - loss: 2.3569
Epoch 3/150
18727/18727 [==============================] - 9s 470us/step - loss: 2.2113
Epoch 4/150
18727/18727 [==============================] - 9s 472us/step - loss: 2.1107
Epoch 5/150
18727/18727 [==============================] - 9s 470us/step - loss: 2.0345
Epoch 6/150
18727/18727 [==============================] - 9s 470us/step - loss: 1.9677
Epoch 7/150
18727/18727 [==============================] - 9s 469us/step - loss: 1.8936
Epoch 8/150
18727/18727 [==============================] - 9s 471us/step - loss: 1.8304
Epoch 9/150
18727/18727 [==============================] - 9

In [13]:
# word based
temp_word = [1.5, 0.75, 0.25]
epochs_num = [50]
error_in_word_list_1, new_test_word_list_1 =  running(filepath, temp_word, epochs_num)

Epoch 1/50
11235/11235 [==============================] - 6s 563us/step - loss: 6.1668
Epoch 2/50
11235/11235 [==============================] - 6s 500us/step - loss: 5.7010
Epoch 3/50
11235/11235 [==============================] - 6s 502us/step - loss: 5.4229
Epoch 4/50
11235/11235 [==============================] - 6s 503us/step - loss: 5.2123
Epoch 5/50
11235/11235 [==============================] - 6s 501us/step - loss: 5.0144
Epoch 6/50
11235/11235 [==============================] - 6s 500us/step - loss: 4.8211
Epoch 7/50
11235/11235 [==============================] - 6s 503us/step - loss: 4.5780
Epoch 8/50
11235/11235 [==============================] - 6s 505us/step - loss: 4.2930
Epoch 9/50
11235/11235 [==============================] - 6s 498us/step - loss: 3.9490
Epoch 10/50
11235/11235 [==============================] - 6s 502us/step - loss: 3.5274
Epoch 11/50
11235/11235 [==============================] - 6s 504us/step - loss: 3.0630
Epoch 12/50
11235/11235 [================

In [14]:
# word based
temp_word = [1.5, 0.75, 0.25]
epochs_num = [100]
error_in_word_list_2, new_test_word_list_2 =  running(filepath, temp_word, epochs_num)

Epoch 1/100
11235/11235 [==============================] - 7s 582us/step - loss: 6.1682
Epoch 2/100
11235/11235 [==============================] - 6s 504us/step - loss: 5.7347
Epoch 3/100
11235/11235 [==============================] - 6s 502us/step - loss: 5.4695
Epoch 4/100
11235/11235 [==============================] - 6s 500us/step - loss: 5.2307
Epoch 5/100
11235/11235 [==============================] - 6s 502us/step - loss: 5.0352
Epoch 6/100
11235/11235 [==============================] - 6s 507us/step - loss: 4.8212
Epoch 7/100
11235/11235 [==============================] - 6s 505us/step - loss: 4.5574
Epoch 8/100
11235/11235 [==============================] - 6s 500us/step - loss: 4.2519
Epoch 9/100
11235/11235 [==============================] - 6s 503us/step - loss: 3.8650
Epoch 10/100
11235/11235 [==============================] - 6s 503us/step - loss: 3.4047
Epoch 11/100
11235/11235 [==============================] - 6s 502us/step - loss: 2.8981
Epoch 12/100
11235/11235 [====

In [15]:
# word based
temp_word = [1.5, 0.75, 0.25]
epochs_num = [150]
error_in_word_list_3, new_test_word_list_3 =  running(filepath, temp_word, epochs_num)

Epoch 1/150
11235/11235 [==============================] - 9s 808us/step - loss: 6.1619
Epoch 2/150
11235/11235 [==============================] - 6s 528us/step - loss: 5.6870
Epoch 3/150
11235/11235 [==============================] - 6s 534us/step - loss: 5.3983
Epoch 4/150
11235/11235 [==============================] - 6s 534us/step - loss: 5.1924
Epoch 5/150
11235/11235 [==============================] - 6s 535us/step - loss: 4.9896
Epoch 6/150
11235/11235 [==============================] - 6s 534us/step - loss: 4.7756
Epoch 7/150
11235/11235 [==============================] - 6s 531us/step - loss: 4.5245
Epoch 8/150
11235/11235 [==============================] - 6s 526us/step - loss: 4.2174
Epoch 9/150
11235/11235 [==============================] - 6s 535us/step - loss: 3.8332
Epoch 10/150
11235/11235 [==============================] - 6s 539us/step - loss: 3.3736
Epoch 11/150
11235/11235 [==============================] - 6s 534us/step - loss: 2.8564
Epoch 12/150
11235/11235 [====

In [0]:
temp_1 = [1.5]
epochs_num = [20, 100, 150]
error_in_word_list_1, new_test_word_list_1 =  running(filepath, temp_1, epochs_num)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch 1/20
22469/22469 [==============================] - 16s 707us/step - loss: 6.0396
Epoch 2/20
22469/22469 [==============================] - 13s 581us/step - loss: 5.4690
Epoch 3/20
22469/22469 [==============================] - 13s 570us/step - loss: 5.2170
Epoch 4/20
22469/22469 [==============================] - 12s 530us/step - loss: 4.9917
Epoch 5/20
22469/22469 [==============================] - 12s 541us/step - loss: 4.7349
Epoch 6/20
22469/22469 [==============================] - 13s 564us/step - loss: 4.4350
Epoch 7/20
22469/22469 [==============================] - 13s 591us/step - loss: 4.0850
Epoch 8/20
22469/22469 [==============================] - 12s 533us/step - loss: 3.6942
Epoch 9/20
22469/22469 [==============================] - 12s

In [0]:
temp_2 = [0.75]
epochs_num = [60]
error_in_word_list_2, new_test_word_list_2 =  running(filepath, temp_2, epochs_num)

Epoch 1/60
11235/11235 [==============================] - 8s 699us/step - loss: 6.1700
Epoch 2/60
11235/11235 [==============================] - 6s 511us/step - loss: 5.7306
Epoch 3/60
11235/11235 [==============================] - 6s 528us/step - loss: 5.4627
Epoch 4/60
11235/11235 [==============================] - 6s 535us/step - loss: 5.2238
Epoch 5/60
11235/11235 [==============================] - 6s 537us/step - loss: 5.0270
Epoch 6/60
11235/11235 [==============================] - 6s 532us/step - loss: 4.8082
Epoch 7/60
11235/11235 [==============================] - 6s 538us/step - loss: 4.5652
Epoch 8/60
11235/11235 [==============================] - 6s 537us/step - loss: 4.2568
Epoch 9/60
11235/11235 [==============================] - 6s 572us/step - loss: 3.8852
Epoch 10/60
11235/11235 [==============================] - 6s 503us/step - loss: 3.4423
Epoch 11/60
11235/11235 [==============================] - 6s 530us/step - loss: 2.9354
Epoch 12/60
11235/11235 [================

In [0]:
temp_3 = [0.25]
epochs_num = [60]
error_in_word_list_3, new_test_word_list_3 =  running(filepath, temp_3, epochs_num)

Epoch 1/60
11235/11235 [==============================] - 7s 627us/step - loss: 6.1613
Epoch 2/60
11235/11235 [==============================] - 6s 564us/step - loss: 5.7215
Epoch 3/60
11235/11235 [==============================] - 6s 573us/step - loss: 5.4401
Epoch 4/60
11235/11235 [==============================] - 6s 575us/step - loss: 5.2186
Epoch 5/60
11235/11235 [==============================] - 6s 556us/step - loss: 5.0269
Epoch 6/60
11235/11235 [==============================] - 7s 603us/step - loss: 4.8177
Epoch 7/60
11235/11235 [==============================] - 6s 527us/step - loss: 4.5803
Epoch 8/60
11235/11235 [==============================] - 6s 547us/step - loss: 4.2813
Epoch 9/60
11235/11235 [==============================] - 6s 546us/step - loss: 3.9126
Epoch 10/60
11235/11235 [==============================] - 6s 565us/step - loss: 3.4663
Epoch 11/60
11235/11235 [==============================] - 6s 562us/step - loss: 2.9697
Epoch 12/60
11235/11235 [================

In [0]:
filepath = '/content/drive/colab/CS155_MiniProject_3/project3/data/shakespeare.txt'
#temp = [1.5, 0.75, 0.25]
#epochs_num = [100, 150, 200]
#num_words = 200
temp = [1.5]
epochs_num = [50]
for epoch in epochs_num:
    for t in temp:
        error_in_word, new_text_word = rnn_word(filepath, t, epoch)
        #error_in, new_text = rnn(filepath, t, epoch)
        print("************************* Final *************************")
        print("Current epoch number = ", epoch)
        print("Current Temperature = ", t)
        print("Error_In_Word = ", error_in_word, ", Perplexity metrics = ", np.exp(error_in_word))
        #print("Error_In_Char = ", error_in, "Perplexity metrics = ", np.exp(error_in))
        print("************************* Word *************************")
        print("new text word: ", new_text_word)
#         print("************************* Char *************************")
#         print("new text: ", new_text)
#         print("************************* Final End *************************")

{'lively': 0, "nature's": 1, 'refigured': 2, 'wound': 3, 'exceeded': 4, 'arise': 5, "store's": 6, 'gaudy': 7, 'love-suit': 8, 'tendered': 9, 'increasing': 10, 'distraction': 11, 'guides': 12, 'excusing': 13, 'judgement': 14, 'heavy': 15, 'fast': 16, 'drops': 17, 'decay': 18, 'condemned': 19, 'wrong': 20, 'fears': 21, 'gladly': 22, 'short-numbered': 23, 'thoughts': 24, 'abuse': 25, 'fierce': 26, 'bitterness': 27, 'receivest': 28, 'maintain': 29, 'words': 30, 'worthy': 31, 'suborned': 32, 'frown': 33, 'faster': 34, 'breathers': 35, 'limit': 36, 'hues': 37, 'jewels': 38, 'went': 39, 'imitated': 40, "ne'er": 41, 'built': 42, 'i': 43, 'mightst': 44, 'blow': 45, 'scythe': 46, "tongue's": 47, 'spring': 48, 'love-god': 49, 'outstripped': 50, 'rights': 51, 'vices': 52, 'shapes': 53, 'seemly': 54, 'paying': 55, 'cast': 56, 'pity': 57, 'resort': 58, 'penance': 59, 'pursuing': 60, 'bones': 61, 'presage': 62, 'too': 63, 'chase': 64, 'speechless': 65, 'slander': 66, 'wild': 67, "mother's": 68, 'prog

In [0]:
filepath = '/content/drive/colab/CS155_MiniProject_3/project3/data/shakespeare.txt'
#temp = [1.5, 0.75, 0.25]
#epochs_num = [100, 150, 200]
#num_words = 200
temp = [1.5]
epochs_num = [100]
for epoch in epochs_num:
    for t in temp:
        error_in_word, new_text_word = rnn_word(filepath, t, epoch)
        #error_in, new_text = rnn(filepath, t, epoch)
        print("************************* Final *************************")
        print("Current epoch number = ", epoch)
        print("Current Temperature = ", t)
        print("Error_In_Word = ", error_in_word, ", Perplexity metrics = ", np.exp(error_in_word))
        #print("Error_In_Char = ", error_in, "Perplexity metrics = ", np.exp(error_in))
        print("************************* Word *************************")
        print("new text word: ", new_text_word)
#         print("************************* Char *************************")
#         print("new text: ", new_text)
#         print("************************* Final End *************************")

{'lively': 0, "nature's": 1, 'refigured': 2, 'wound': 3, 'exceeded': 4, 'arise': 5, "store's": 6, 'gaudy': 7, 'love-suit': 8, 'tendered': 9, 'increasing': 10, 'distraction': 11, 'guides': 12, 'excusing': 13, 'judgement': 14, 'heavy': 15, 'fast': 16, 'drops': 17, 'decay': 18, 'condemned': 19, 'wrong': 20, 'fears': 21, 'gladly': 22, 'short-numbered': 23, 'thoughts': 24, 'abuse': 25, 'fierce': 26, 'bitterness': 27, 'receivest': 28, 'maintain': 29, 'words': 30, 'worthy': 31, 'suborned': 32, 'frown': 33, 'faster': 34, 'breathers': 35, 'limit': 36, 'hues': 37, 'jewels': 38, 'went': 39, 'imitated': 40, "ne'er": 41, 'built': 42, 'i': 43, 'mightst': 44, 'blow': 45, 'scythe': 46, "tongue's": 47, 'spring': 48, 'love-god': 49, 'outstripped': 50, 'rights': 51, 'vices': 52, 'shapes': 53, 'seemly': 54, 'paying': 55, 'cast': 56, 'pity': 57, 'resort': 58, 'penance': 59, 'pursuing': 60, 'bones': 61, 'presage': 62, 'too': 63, 'chase': 64, 'speechless': 65, 'slander': 66, 'wild': 67, "mother's": 68, 'prog

In [0]:
filepath = '/content/drive/colab/CS155_MiniProject_3/project3/data/shakespeare.txt'
#temp = [1.5, 0.75, 0.25]
#epochs_num = [100, 150, 200]
#num_words = 200
temp = [1.5]
epochs_num = [150]
for epoch in epochs_num:
    for t in temp:
        error_in_word, new_text_word = rnn_word(filepath, t, epoch)
        #error_in, new_text = rnn(filepath, t, epoch)
        print("************************* Final *************************")
        print("Current epoch number = ", epoch)
        print("Current Temperature = ", t)
        print("Error_In_Word = ", error_in_word, ", Perplexity metrics = ", np.exp(error_in_word))
        #print("Error_In_Char = ", error_in, "Perplexity metrics = ", np.exp(error_in))
        print("************************* Word *************************")
        print("new text word: ", new_text_word)
#         print("************************* Char *************************")
#         print("new text: ", new_text)
#         print("************************* Final End *************************")

{'lively': 0, "nature's": 1, 'refigured': 2, 'wound': 3, 'exceeded': 4, 'arise': 5, "store's": 6, 'gaudy': 7, 'love-suit': 8, 'tendered': 9, 'increasing': 10, 'distraction': 11, 'guides': 12, 'excusing': 13, 'judgement': 14, 'heavy': 15, 'fast': 16, 'drops': 17, 'decay': 18, 'condemned': 19, 'wrong': 20, 'fears': 21, 'gladly': 22, 'short-numbered': 23, 'thoughts': 24, 'abuse': 25, 'fierce': 26, 'bitterness': 27, 'receivest': 28, 'maintain': 29, 'words': 30, 'worthy': 31, 'suborned': 32, 'frown': 33, 'faster': 34, 'breathers': 35, 'limit': 36, 'hues': 37, 'jewels': 38, 'went': 39, 'imitated': 40, "ne'er": 41, 'built': 42, 'i': 43, 'mightst': 44, 'blow': 45, 'scythe': 46, "tongue's": 47, 'spring': 48, 'love-god': 49, 'outstripped': 50, 'rights': 51, 'vices': 52, 'shapes': 53, 'seemly': 54, 'paying': 55, 'cast': 56, 'pity': 57, 'resort': 58, 'penance': 59, 'pursuing': 60, 'bones': 61, 'presage': 62, 'too': 63, 'chase': 64, 'speechless': 65, 'slander': 66, 'wild': 67, "mother's": 68, 'prog

In [0]:
filepath = '/content/drive/colab/CS155_MiniProject_3/project3/data/shakespeare.txt'
#temp = [1.5, 0.75, 0.25]
#epochs_num = [100, 150, 200]
#num_words = 300
temp = [1.5]
epochs_num = [200]
for epoch in epochs_num:
    for t in temp:
        error_in_word, new_text_word = rnn_word(filepath, t, epoch)
        #error_in, new_text = rnn(filepath, t, epoch)
        print("************************* Final *************************")
        print("Current epoch number = ", epoch)
        print("Current Temperature = ", t)
        print("Error_In_Word = ", error_in_word, ", Perplexity metrics = ", np.exp(error_in_word))
        #print("Error_In_Char = ", error_in, "Perplexity metrics = ", np.exp(error_in))
        print("************************* Word *************************")
        print("new text word: ", new_text_word)
#         print("************************* Char *************************")
#         print("new text: ", new_text)
#         print("************************* Final End *************************")

{'sight': 0, 'shook': 1, 'stole': 2, 'reigned': 3, 'plea': 4, "'thus": 5, 'cheek': 6, 'bath': 7, 'so': 8, 'addition': 9, 'dear': 10, 'latch': 11, 'titles': 12, 'abuses': 13, '?': 14, 'yellow': 15, 'bail': 16, 'profit': 17, 'bear': 18, 'nights': 19, "picture's": 20, 're-survey': 21, 'cloak': 22, 'tenure': 23, 'beck': 24, "who's": 25, 'gains': 26, 'bright': 27, 'seek': 28, 'sepulchres': 29, 'day': 30, 'afloat': 31, 'intermixed': 32, 'touched': 33, 'promise': 34, 'brightness': 35, 'sunset': 36, 'twilight': 37, 'pretty': 38, "love's": 39, "lion's": 40, 'what': 41, 'many': 42, 'kind-hearted': 43, 'assure': 44, 'death-bed': 45, 'hid': 46, 'bends': 47, 'grow': 48, 'grew': 49, 'thinly': 50, 'barren': 51, 'cold': 52, 'committed': 53, 'fountains': 54, 'carved': 55, 'end': 56, 'none': 57, 'straight': 58, 'thanks': 59, 'reasons': 60, 'trees': 61, 'weak': 62, 'minds': 63, 'seal': 64, 'big': 65, 'rigour': 66, 'clock': 67, 'sleep': 68, 'aspect': 69, 'vow': 70, 'tied': 71, 'sake': 72, 'journey': 73, '

In [0]:
filepath = '/content/drive/colab/CS155_MiniProject_3/project3/data/shakespeare.txt'
#temp = [1.5, 0.75, 0.25]
#epochs_num = [100, 150, 200]
#num_words = 300
temp = [0.75]
epochs_num = [50]
for epoch in epochs_num:
    for t in temp:
        error_in_word, new_text_word = rnn_word(filepath, t, epoch)
        #error_in, new_text = rnn(filepath, t, epoch)
        print("************************* Final *************************")
        print("Current epoch number = ", epoch)
        print("Current Temperature = ", t)
        print("Error_In_Word = ", error_in_word, ", Perplexity metrics = ", np.exp(error_in_word))
        #print("Error_In_Char = ", error_in, "Perplexity metrics = ", np.exp(error_in))
        print("************************* Word *************************")
        print("new text word: ", new_text_word)
#         print("************************* Char *************************")
#         print("new text: ", new_text)
#         print("************************* Final End *************************")

{'sight': 0, 'shook': 1, 'stole': 2, 'reigned': 3, 'plea': 4, "'thus": 5, 'cheek': 6, 'bath': 7, 'so': 8, 'addition': 9, 'dear': 10, 'latch': 11, 'titles': 12, 'abuses': 13, '?': 14, 'yellow': 15, 'bail': 16, 'profit': 17, 'bear': 18, 'nights': 19, "picture's": 20, 're-survey': 21, 'cloak': 22, 'tenure': 23, 'beck': 24, "who's": 25, 'gains': 26, 'bright': 27, 'seek': 28, 'sepulchres': 29, 'day': 30, 'afloat': 31, 'intermixed': 32, 'touched': 33, 'promise': 34, 'brightness': 35, 'sunset': 36, 'twilight': 37, 'pretty': 38, "love's": 39, "lion's": 40, 'what': 41, 'many': 42, 'kind-hearted': 43, 'assure': 44, 'death-bed': 45, 'hid': 46, 'bends': 47, 'grow': 48, 'grew': 49, 'thinly': 50, 'barren': 51, 'cold': 52, 'committed': 53, 'fountains': 54, 'carved': 55, 'end': 56, 'none': 57, 'straight': 58, 'thanks': 59, 'reasons': 60, 'trees': 61, 'weak': 62, 'minds': 63, 'seal': 64, 'big': 65, 'rigour': 66, 'clock': 67, 'sleep': 68, 'aspect': 69, 'vow': 70, 'tied': 71, 'sake': 72, 'journey': 73, '

In [0]:
filepath = '/content/drive/colab/CS155_MiniProject_3/project3/data/shakespeare.txt'
#temp = [1.5, 0.75, 0.25]
#epochs_num = [100, 150, 200]
#num_words = 300
temp = [0.75]
epochs_num = [100]
for epoch in epochs_num:
    for t in temp:
        error_in_word, new_text_word = rnn_word(filepath, t, epoch)
        #error_in, new_text = rnn(filepath, t, epoch)
        print("************************* Final *************************")
        print("Current epoch number = ", epoch)
        print("Current Temperature = ", t)
        print("Error_In_Word = ", error_in_word, ", Perplexity metrics = ", np.exp(error_in_word))
        #print("Error_In_Char = ", error_in, "Perplexity metrics = ", np.exp(error_in))
        print("************************* Word *************************")
        print("new text word: ", new_text_word)
#         print("************************* Char *************************")
#         print("new text: ", new_text)
#         print("************************* Final End *************************")

{'sight': 0, 'shook': 1, 'stole': 2, 'reigned': 3, 'plea': 4, "'thus": 5, 'cheek': 6, 'bath': 7, 'so': 8, 'addition': 9, 'dear': 10, 'latch': 11, 'titles': 12, 'abuses': 13, '?': 14, 'yellow': 15, 'bail': 16, 'profit': 17, 'bear': 18, 'nights': 19, "picture's": 20, 're-survey': 21, 'cloak': 22, 'tenure': 23, 'beck': 24, "who's": 25, 'gains': 26, 'bright': 27, 'seek': 28, 'sepulchres': 29, 'day': 30, 'afloat': 31, 'intermixed': 32, 'touched': 33, 'promise': 34, 'brightness': 35, 'sunset': 36, 'twilight': 37, 'pretty': 38, "love's": 39, "lion's": 40, 'what': 41, 'many': 42, 'kind-hearted': 43, 'assure': 44, 'death-bed': 45, 'hid': 46, 'bends': 47, 'grow': 48, 'grew': 49, 'thinly': 50, 'barren': 51, 'cold': 52, 'committed': 53, 'fountains': 54, 'carved': 55, 'end': 56, 'none': 57, 'straight': 58, 'thanks': 59, 'reasons': 60, 'trees': 61, 'weak': 62, 'minds': 63, 'seal': 64, 'big': 65, 'rigour': 66, 'clock': 67, 'sleep': 68, 'aspect': 69, 'vow': 70, 'tied': 71, 'sake': 72, 'journey': 73, '

In [0]:
filepath = '/content/drive/colab/CS155_MiniProject_3/project3/data/shakespeare.txt'
#temp = [1.5, 0.75, 0.25]
#epochs_num = [100, 150, 200]
#num_words = 300
temp = [0.75]
epochs_num = [150]
for epoch in epochs_num:
    for t in temp:
        error_in_word, new_text_word = rnn_word(filepath, t, epoch)
        #error_in, new_text = rnn(filepath, t, epoch)
        print("************************* Final *************************")
        print("Current epoch number = ", epoch)
        print("Current Temperature = ", t)
        print("Error_In_Word = ", error_in_word, ", Perplexity metrics = ", np.exp(error_in_word))
        #print("Error_In_Char = ", error_in, "Perplexity metrics = ", np.exp(error_in))
        print("************************* Word *************************")
        print("new text word: ", new_text_word)
#         print("************************* Char *************************")
#         print("new text: ", new_text)
#         print("************************* Final End *************************")

{'sight': 0, 'shook': 1, 'stole': 2, 'reigned': 3, 'plea': 4, "'thus": 5, 'cheek': 6, 'bath': 7, 'so': 8, 'addition': 9, 'dear': 10, 'latch': 11, 'titles': 12, 'abuses': 13, '?': 14, 'yellow': 15, 'bail': 16, 'profit': 17, 'bear': 18, 'nights': 19, "picture's": 20, 're-survey': 21, 'cloak': 22, 'tenure': 23, 'beck': 24, "who's": 25, 'gains': 26, 'bright': 27, 'seek': 28, 'sepulchres': 29, 'day': 30, 'afloat': 31, 'intermixed': 32, 'touched': 33, 'promise': 34, 'brightness': 35, 'sunset': 36, 'twilight': 37, 'pretty': 38, "love's": 39, "lion's": 40, 'what': 41, 'many': 42, 'kind-hearted': 43, 'assure': 44, 'death-bed': 45, 'hid': 46, 'bends': 47, 'grow': 48, 'grew': 49, 'thinly': 50, 'barren': 51, 'cold': 52, 'committed': 53, 'fountains': 54, 'carved': 55, 'end': 56, 'none': 57, 'straight': 58, 'thanks': 59, 'reasons': 60, 'trees': 61, 'weak': 62, 'minds': 63, 'seal': 64, 'big': 65, 'rigour': 66, 'clock': 67, 'sleep': 68, 'aspect': 69, 'vow': 70, 'tied': 71, 'sake': 72, 'journey': 73, '

In [0]:
filepath = '/content/drive/colab/CS155_MiniProject_3/project3/data/shakespeare.txt'
#temp = [1.5, 0.75, 0.25]
#epochs_num = [100, 150, 200]
#num_words = 300
temp = [0.75]
epochs_num = [200]
for epoch in epochs_num:
    for t in temp:
        error_in_word, new_text_word = rnn_word(filepath, t, epoch)
        #error_in, new_text = rnn(filepath, t, epoch)
        print("************************* Final *************************")
        print("Current epoch number = ", epoch)
        print("Current Temperature = ", t)
        print("Error_In_Word = ", error_in_word, ", Perplexity metrics = ", np.exp(error_in_word))
        #print("Error_In_Char = ", error_in, "Perplexity metrics = ", np.exp(error_in))
        print("************************* Word *************************")
        print("new text word: ", new_text_word)
#         print("************************* Char *************************")
#         print("new text: ", new_text)
#         print("************************* Final End *************************")

{'sight': 0, 'shook': 1, 'stole': 2, 'reigned': 3, 'plea': 4, "'thus": 5, 'cheek': 6, 'bath': 7, 'so': 8, 'addition': 9, 'dear': 10, 'latch': 11, 'titles': 12, 'abuses': 13, '?': 14, 'yellow': 15, 'bail': 16, 'profit': 17, 'bear': 18, 'nights': 19, "picture's": 20, 're-survey': 21, 'cloak': 22, 'tenure': 23, 'beck': 24, "who's": 25, 'gains': 26, 'bright': 27, 'seek': 28, 'sepulchres': 29, 'day': 30, 'afloat': 31, 'intermixed': 32, 'touched': 33, 'promise': 34, 'brightness': 35, 'sunset': 36, 'twilight': 37, 'pretty': 38, "love's": 39, "lion's": 40, 'what': 41, 'many': 42, 'kind-hearted': 43, 'assure': 44, 'death-bed': 45, 'hid': 46, 'bends': 47, 'grow': 48, 'grew': 49, 'thinly': 50, 'barren': 51, 'cold': 52, 'committed': 53, 'fountains': 54, 'carved': 55, 'end': 56, 'none': 57, 'straight': 58, 'thanks': 59, 'reasons': 60, 'trees': 61, 'weak': 62, 'minds': 63, 'seal': 64, 'big': 65, 'rigour': 66, 'clock': 67, 'sleep': 68, 'aspect': 69, 'vow': 70, 'tied': 71, 'sake': 72, 'journey': 73, '

In [0]:
filepath = '/content/drive/colab/CS155_MiniProject_3/project3/data/shakespeare.txt'
#temp = [1.5, 0.75, 0.25]
#epochs_num = [100, 150, 200]
#num_words = 300
temp = [0.25]
epochs_num = [50]
for epoch in epochs_num:
    for t in temp:
        error_in_word, new_text_word = rnn_word(filepath, t, epoch)
        #error_in, new_text = rnn(filepath, t, epoch)
        print("************************* Final *************************")
        print("Current epoch number = ", epoch)
        print("Current Temperature = ", t)
        print("Error_In_Word = ", error_in_word, ", Perplexity metrics = ", np.exp(error_in_word))
        #print("Error_In_Char = ", error_in, "Perplexity metrics = ", np.exp(error_in))
        print("************************* Word *************************")
        print("new text word: ", new_text_word)
#         print("************************* Char *************************")
#         print("new text: ", new_text)
#         print("************************* Final End *************************")

{'sight': 0, 'shook': 1, 'stole': 2, 'reigned': 3, 'plea': 4, "'thus": 5, 'cheek': 6, 'bath': 7, 'so': 8, 'addition': 9, 'dear': 10, 'latch': 11, 'titles': 12, 'abuses': 13, '?': 14, 'yellow': 15, 'bail': 16, 'profit': 17, 'bear': 18, 'nights': 19, "picture's": 20, 're-survey': 21, 'cloak': 22, 'tenure': 23, 'beck': 24, "who's": 25, 'gains': 26, 'bright': 27, 'seek': 28, 'sepulchres': 29, 'day': 30, 'afloat': 31, 'intermixed': 32, 'touched': 33, 'promise': 34, 'brightness': 35, 'sunset': 36, 'twilight': 37, 'pretty': 38, "love's": 39, "lion's": 40, 'what': 41, 'many': 42, 'kind-hearted': 43, 'assure': 44, 'death-bed': 45, 'hid': 46, 'bends': 47, 'grow': 48, 'grew': 49, 'thinly': 50, 'barren': 51, 'cold': 52, 'committed': 53, 'fountains': 54, 'carved': 55, 'end': 56, 'none': 57, 'straight': 58, 'thanks': 59, 'reasons': 60, 'trees': 61, 'weak': 62, 'minds': 63, 'seal': 64, 'big': 65, 'rigour': 66, 'clock': 67, 'sleep': 68, 'aspect': 69, 'vow': 70, 'tied': 71, 'sake': 72, 'journey': 73, '

In [0]:
filepath = '/content/drive/colab/CS155_MiniProject_3/project3/data/shakespeare.txt'
#temp = [1.5, 0.75, 0.25]
#epochs_num = [100, 150, 200]
#num_words = 300
temp = [0.25]
epochs_num = [100]
for epoch in epochs_num:
    for t in temp:
        error_in_word, new_text_word = rnn_word(filepath, t, epoch)
        #error_in, new_text = rnn(filepath, t, epoch)
        print("************************* Final *************************")
        print("Current epoch number = ", epoch)
        print("Current Temperature = ", t)
        print("Error_In_Word = ", error_in_word, ", Perplexity metrics = ", np.exp(error_in_word))
        #print("Error_In_Char = ", error_in, "Perplexity metrics = ", np.exp(error_in))
        print("************************* Word *************************")
        print("new text word: ", new_text_word)
#         print("************************* Char *************************")
#         print("new text: ", new_text)
#         print("************************* Final End *************************")

{'sight': 0, 'shook': 1, 'stole': 2, 'reigned': 3, 'plea': 4, "'thus": 5, 'cheek': 6, 'bath': 7, 'so': 8, 'addition': 9, 'dear': 10, 'latch': 11, 'titles': 12, 'abuses': 13, '?': 14, 'yellow': 15, 'bail': 16, 'profit': 17, 'bear': 18, 'nights': 19, "picture's": 20, 're-survey': 21, 'cloak': 22, 'tenure': 23, 'beck': 24, "who's": 25, 'gains': 26, 'bright': 27, 'seek': 28, 'sepulchres': 29, 'day': 30, 'afloat': 31, 'intermixed': 32, 'touched': 33, 'promise': 34, 'brightness': 35, 'sunset': 36, 'twilight': 37, 'pretty': 38, "love's": 39, "lion's": 40, 'what': 41, 'many': 42, 'kind-hearted': 43, 'assure': 44, 'death-bed': 45, 'hid': 46, 'bends': 47, 'grow': 48, 'grew': 49, 'thinly': 50, 'barren': 51, 'cold': 52, 'committed': 53, 'fountains': 54, 'carved': 55, 'end': 56, 'none': 57, 'straight': 58, 'thanks': 59, 'reasons': 60, 'trees': 61, 'weak': 62, 'minds': 63, 'seal': 64, 'big': 65, 'rigour': 66, 'clock': 67, 'sleep': 68, 'aspect': 69, 'vow': 70, 'tied': 71, 'sake': 72, 'journey': 73, '

In [0]:
filepath = '/content/drive/colab/CS155_MiniProject_3/project3/data/shakespeare.txt'
#temp = [1.5, 0.75, 0.25]
#epochs_num = [100, 150, 200]
#num_words = 300
temp = [0.25]
epochs_num = [150]
for epoch in epochs_num:
    for t in temp:
        error_in_word, new_text_word = rnn_word(filepath, t, epoch)
        #error_in, new_text = rnn(filepath, t, epoch)
        print("************************* Final *************************")
        print("Current epoch number = ", epoch)
        print("Current Temperature = ", t)
        print("Error_In_Word = ", error_in_word, ", Perplexity metrics = ", np.exp(error_in_word))
        #print("Error_In_Char = ", error_in, "Perplexity metrics = ", np.exp(error_in))
        print("************************* Word *************************")
        print("new text word: ", new_text_word)
#         print("************************* Char *************************")
#         print("new text: ", new_text)
#         print("************************* Final End *************************")

In [0]:
filepath = '/content/drive/colab/CS155_MiniProject_3/project3/data/shakespeare.txt'
#temp = [1.5, 0.75, 0.25]
#epochs_num = [100, 150, 200]
#num_words = 300
temp = [0.25]
epochs_num = [200]
for epoch in epochs_num:
    for t in temp:
        error_in_word, new_text_word = rnn_word(filepath, t, epoch)
        #error_in, new_text = rnn(filepath, t, epoch)
        print("************************* Final *************************")
        print("Current epoch number = ", epoch)
        print("Current Temperature = ", t)
        print("Error_In_Word = ", error_in_word, ", Perplexity metrics = ", np.exp(error_in_word))
        #print("Error_In_Char = ", error_in, "Perplexity metrics = ", np.exp(error_in))
        print("************************* Word *************************")
        print("new text word: ", new_text_word)
#         print("************************* Char *************************")
#         print("new text: ", new_text)
#         print("************************* Final End *************************")